# Entity Matcher in production - contextualize time series to assets using Cognite Functions

For every new customer, we ingest their data and perform some sort of contextualization by mapping time series to assets. Quite often, there is not an identical match between a field on the time series to e.g. the asset name, but a somewhat similar name. The most typical example we use in Cognite is the asset `21PT1019` and the time series `IAA_21PT1019.PV` or similar. In our contextualization toolbox (an [API](https://docs.cognite.com/api/playground/#operation/entityMatchingFit) with a corresponding [https://cognite-sdk-experimental.readthedocs-hosted.com/en/latest/cognite.html#contextualization](SDK)), we have a tool to solve this problem called entity matching. Entity matching means joining two datasets one a common key with fuzzyness, and our implementation is a machine learning model that can be trained supervised or unsupervised. 

In this tutorial, we'll take the time series and assets from the `publicdata` tenant, and deploy a Cognite Function that performs entity matching to map the time series to assets, and schedule it so it runs periodically. This can be used out of the box for many customers as an initial contextualization step. 

In [ ]:
from getpass import getpass
from cognite.experimental import CogniteClient

In [ ]:
api_key = getpass()
client = CogniteClient(
    api_key=api_key,
    project="functions-tutorial",
    client_name="DSHub",
    base_url="https://greenfield.cognitedata.com"
)

## Defining our function
We now define the function we wish to deploy. In this simple example, we define the function within the notebook itself. This is convenient to get started quickly, but for larger functions, you would probably define the function in one or more separate files, or use the CI/CD in Github or Gitlab.

Note that the function needs to be named `handle`, and the arguments to the function must be a subset of `(client, data, secrets)`. In this example we only need `client` and `data`. The `client` argument is an instance of a `CogniteClient`, which will be provided to us automatically when the function is deployed as long as we give an `api_key` when creating the function. The data we give when calling the function, is passed through the `data` argument.

In [ ]:
def handle(client, data):
    # When deploying a function from a notebook like this, all imports must be performed inside the `handle` function.
    from cognite.experimental import CogniteClient
    from cognite.client.data_classes import TimeSeriesUpdate
    import time
    
    # The entity matcher suggests matches with a certain score. To achieve a reasonable result, this score must be adjusted. 
    # The default value of 0.75 has been chosen by inspecting the outcome of this function, and may be different on data from other customers.
    good_match_threshold = data.get("good_match_threshold", 0.75)
    
    # Create experimental SDK client as the contextualization API's are in playground and are thus not available in the regular SDK.
    client = CogniteClient(api_key = client.config.api_key, base_url = client.config.base_url, project = client.config.project)

    # Download all assets and time series, using 5 requests in parallel
    assets = client.assets.list(limit=-1, partitions=5)
    time_series = client.time_series.list(limit=-1, partitions=5)
    
    # Create simplified objects with only name and id
    assets_simplified = [{"id": asset.id, "name": asset.name} for asset in assets]
    time_series_simplified = [{"id": ts.id, "name": ts.name} for ts in time_series]

    # Train the ML Entity Matcher on the data. The SDK expects as input the array of objects you match FROM (time series) and a list of what you match TO (assets)
    t0 = time.time()
    model = client.entity_matching.fit_ml(match_from = time_series_simplified, match_to = assets_simplified)
    print(f"Training entity matcher model with id {model.model_id} ...")
    model.wait_for_completion()
    t1 = time.time()
    print(f"Model {model.model_id} trained on {len(assets_simplified)} assets and {len(time_series_simplified)} time series using {t1-t0} seconds")

    # Use the ML Entity Matcher model to match the data. This model can be reused, so training is not necessary each time, but we do it for simplicity in this example.
    t0 = time.time()
    job = model.predict_ml(time_series_simplified)
    result = job.result # This will wait for completion
    t1 = time.time()
    print(f"Predict finished after {t1-t0} seconds on {len(time_series_simplified)} time series.")
    
    # Filter out the best matches with the threshold specified in the input
    good_match_count = 0
    time_series_updates = []
    for item in result["items"]:
        match_from = item["matchFrom"] # Time series
        matches = item["matches"] # Suggested asset matches for the time series
        good_matches = [match for match in matches if match["score"] >= good_match_threshold]
        if len(good_matches) > 0:
            good_match_count += 1
            best_match = good_matches[0]
            time_series_updates.append(TimeSeriesUpdate(id=match_from["id"]).asset_id.set(best_match["matchTo"]["id"]))
    
    client.time_series.update(time_series_updates) # uncomment to actually update the asset_id field
    print(f"Matched {good_match_count} time series to assets")
    return {
        "matches": good_match_count
    }


Test the function by running it locally. We send in the `CogniteClient` and an empty dictionary as data.

In [ ]:
result = handle(client, {})

## Deploying and calling the function
Now that we have verified that it works, we can create a Cognite Function that performs this on demand or by a schedule. The function code is uploaded to CDF simply by setting the argument `function_handle` equal to `handle`, which is the name of the function we created above.

In [ ]:
name = # Insert your name here

In [ ]:
external_id = f"{name} entity matcher"
function = client.functions.create(
    name = external_id, 
    external_id=external_id, 
    description = "Entity matcher example", 
    api_key = api_key, 
    function_handle = handle
)

In [ ]:
# Run this until it shows status ready
function = client.functions.retrieve(external_id = external_id)
function

Once the function is ready, we can call it. The default behaviour is that `function.call()` will wait until the function call is completed. You can also do `function.call(wait=False)` if you e.g. want to start multiple function calls at the same time.

In [ ]:
function_call = function.call() # This will wait for the function call to complete.
function_call

If the function failed, or you just want to see the logs, you can check the `function_call.get_logs()` function. All print statements and stack traces from errors are shown in the logs.

In [ ]:
function_call.get_logs()

If you returned data from the function (like in the above example), you can access that from the `function_call.get_response()` function. 

In [ ]:
function_call.get_response()

## Creating a schedule
We can schedule the function to call periodically by creating a `schedules` objects. It must point to a function with an external id, and the syntax for when the it is scheduled is unix cron (see https://crontab.guru to play around with values). Time zone for schedules is GMT.

In [ ]:
schedule = client.functions.schedules.create(
    name = f"{name} entity matcher", 
    function_external_id=external_id, 
    cron_expression = "* * * * *"
)

## Cleanup
When we are done, we clean up after ourselves byt deleting the schedule and function.

In [ ]:
client.functions.schedules.delete(id=schedule.id)
client.functions.delete(id=function.id)